In [1]:
#load packages
import pandas as pd
import numpy as np
from collections import defaultdict

In [12]:
#exclude records with special event flags
fn_in1 = open("C:/Users/xiachai/Desktop/KF/event_file.txt", 'r')
fn_out1 = open("C:/Users/xiachai/Desktop/KF/noXJ.txt", 'w')

for line in fn_in1.readlines():
    v = line.strip().split('|')
    if v[3] != "X" and v[3] != "J":
        print >> fn_out1, '%s' % ('|'.join(v[s] for s in range(7)))
        
fn_in1.close()
fn_out1.close()

In [10]:
#exclude conversion only records (optional) 
fn_out2 = open("C:/Users/xiachai/Desktop/KF/event_file_time.txt", 'w')

d = defaultdict(int)
conv_only = set([])

with open("C:/Users/xiachai/Desktop/KF/noXJ.txt", 'r') as fn_in2:
    for line in fn_in2.readlines():
        v = line.strip().split('|')    

        d[v[0]] += 1
    
for key in d:
    if d[key] == 1:
        conv_only.add(key)
        
with open("C:/Users/xiachai/Desktop/KF/noXJ.txt", 'r') as fn_in3:
    for line in fn_in3.readlines():
        v = line.strip().split('|')    
        if v[0] not in conv_only:
            print >> fn_out2, '%s' % (line.strip())
            
        

In [16]:
#read file in pandas, reformat timestamp
fn_in4 = open("C:/Users/xiachai/Desktop/KF/event_file_time.txt", 'r')
data = pd.read_table(fn_in4, delimiter = '|', names = ['id','timestamp','conv_flag','event_type','pub','tactic','campaign_type'])
data['timestamp'] = pd.to_datetime(data['timestamp'].apply(str), format='%Y%m%d%H%M%S')

,id,timestamp,conv_flag,event_type,pub,tactic,campaign_type
0,2588,2016-02-18 10:23:01,NaN,I,AO,PRD,D
1,2588,2016-02-24 10:04:04,NaN,I,AO,PRD,D
2,2588,2016-03-01 10:57:22,NaN,I,AO,PRD,D
3,2588,2016-03-05 13:54:59,1,P,NaN,NaN,NaN
4,2940,2016-02-16 08:24:16,NaN,I,T1,PBT,W


In [6]:
#last touch point to conversion
ltp_conv = data.groupby('id').tail(2).reset_index(drop=True)

ltp_conv['delta_time'] = ltp_conv.groupby('id')['timestamp'].diff()
ltp_conv['delta_time_sec']=ltp_conv['delta_time'] / np.timedelta64(1, 's')

fn_out3 = open("C:/Users/xiachai/Desktop/KF/KF_lasttp_conv.txt",'w')
ltp_conv.to_csv(fn_out3, sep='|',index=False, header=['user_ID','timestamp','converter_flag','event_type','publisher','tactic','campaign_type','delta_time','delta_time_sec'])
fn_out3.close()

In [13]:
#keep first touch point or each id, drop unuseful columns
top = data.groupby('id').head(1).reset_index(drop = True).drop(["conv_flag","event_type","pub","tactic","campaign_type"],axis = 1)
top.head()

,id,timestamp
0,2588,2016-02-18 10:23:01
1,2940,2016-02-16 08:24:16
2,6245_1,2016-01-31 18:44:13
3,7534,2016-02-02 12:57:39
4,9802,2016-01-28 09:55:42


In [14]:
#keep last touch point for each id, drop unuseful columns
bottom = data.groupby('id').tail(1).reset_index(drop = True).drop(["event_type","pub","tactic","campaign_type"],axis = 1)
bottom.head()

,id,timestamp,conv_flag
0,2588,2016-03-05 13:54:59,1
1,2940,2016-02-19 13:28:59,1
2,6245_1,2016-02-27 17:01:59,1
3,7534,2016-02-05 16:49:59,1
4,9802,2016-02-21 12:03:59,1


In [9]:
#merge top and bottom on id
new_table = pd.merge(top,bottom,on = 'id')
new_table.head()

,id,timestamp_x,timestamp_y,conv_flag
0,2588,2016-02-18 10:23:01,2016-03-05 13:54:59,1
1,2940,2016-02-16 08:24:16,2016-02-19 13:28:59,1
2,6245_1,2016-01-31 18:44:13,2016-02-27 17:01:59,1
3,7534,2016-02-02 12:57:39,2016-02-05 16:49:59,1
4,9802,2016-01-28 09:55:42,2016-02-21 12:03:59,1


In [11]:
#calculate time intervals between first touch point and conversion by id, output to a text file
new_table['time_delta'] = new_table["timestamp_y"] - new_table["timestamp_x"]
new_table['time_delta_sec'] = new_table['time_delta']/ np.timedelta64(1, 's')
#new_table.head()
new_table.to_csv("C:/Users/xiachai/Desktop/KF/KF_firsttp_conv.txt",sep='|',index=False)